### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [2]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [3]:
today = date.today()
tomorrow = today + timedelta(days=1)

day = today

In [4]:
data_var = day.strftime('%Y-%m-%d')
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
# ligas = temporada[temporada['Date'] == data_var]['League'].unique().tolist()
# temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos[['Date', 'League', 'Time', 'Home', 'Away', 
       'Odds_H', 'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A', ]]

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A
5437,2023-10-31,Eua Nba,23:00,Phoenix Suns,San Antonio Spurs,1.36,3.25,227.5,1.91,1.91,-6.5,1.91,1.91
5438,2023-10-31,Eua Nba,23:30,Los Angeles Clippers,Orlando Magic,1.32,3.50,220.5,1.91,1.91,-7.5,1.91,1.91
5439,2023-10-31,Europa Euroliga,16:30,Milano,Maccabi Tel Aviv,1.48,2.75,158.5,1.91,1.91,-5.5,1.95,1.87
5440,2023-10-31,Europa Eurocopa,14:30,Cedevita Olimpija,BC Wolves,1.59,2.50,167.5,1.91,1.91,-3.5,1.91,1.91
5441,2023-10-31,Europa Eurocopa,16:00,Trento,Turk Telekom,2.15,1.74,159.5,1.95,1.87,2.5,1.87,1.95
5442,2023-10-31,Europa Eurocopa,16:30,Joventut,Hamburg,1.08,8.50,170.5,1.91,1.91,-16.5,1.91,1.91
5443,2023-10-31,China Cba,08:35,Zhejiang Chouzhou,Ningbo Rockets,1.01,17.50,200.5,1.91,1.83,-20.5,1.87,1.87
5444,2023-10-31,China Cba,09:00,Xinjiang,Qingdao,1.25,4.10,197.5,1.87,1.87,-8.5,1.87,1.87
5445,2023-10-31,Coreia Do Sul Kbl,07:00,Anyang JungKwanJang,Mobis Phoebus,2.25,1.69,158.5,1.91,1.91,2.5,1.91,1.91
5446,2023-10-31,Coreia Do Sul Kbl,07:00,Seoul Thunders,KoGas,2.25,1.69,170.5,2.00,1.83,2.5,1.91,1.91


## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,23:00,Eua Nba,Phoenix Suns,San Antonio Spurs,227.5,1.91,0.7871,Over
1,23:30,Eua Nba,Los Angeles Clippers,Orlando Magic,220.5,1.91,0.7964,Over
2,16:30,Europa Euroliga,Milano,Maccabi Tel Aviv,158.5,1.91,0.7317,Over
3,14:30,Europa Eurocopa,Cedevita Olimpija,BC Wolves,167.5,1.91,0.8194,Over
4,16:00,Europa Eurocopa,Trento,Turk Telekom,159.5,1.95,0.6497,Over
5,16:30,Europa Eurocopa,Joventut,Hamburg,170.5,1.91,0.8677,Over
8,07:00,Coreia Do Sul Kbl,Anyang JungKwanJang,Mobis Phoebus,158.5,1.91,0.8318,Over


### AH Zebra V1 (LR)

In [9]:
# Duplicando o dataset
df_ah_pos_v1 = jogos.copy()

# AH
df_ah_pos_v1.loc[(df_ah_pos_v1['HA_Line']) > 0, 'Zebra'] = 'Casa'
df_ah_pos_v1.loc[(df_ah_pos_v1['HA_Line']) < 0, 'Zebra'] = 'Fora'
df_ah_pos_v1['Zebra'] = df_ah_pos_v1['Zebra'].astype('category')
# Odds
df_ah_pos_v1.loc[(df_ah_pos_v1['Zebra']) == 'Casa', 'Odds'] = df_ah_pos_v1['HA_Odds_H']
df_ah_pos_v1.loc[(df_ah_pos_v1['Zebra']) == 'Fora', 'Odds'] = df_ah_pos_v1['HA_Odds_A']

df_ah_pos_v1.reset_index(drop=True, inplace=True)
df_ah_pos_v1.replace([np.inf, -np.inf], 0, inplace=True)

if not df_ah_pos_v1.empty:
    mdl_ah_pos_v1 = load_model('ML/ah_pos/v1/v1_ah_pos', verbose=False)
    prev = predict_model(mdl_ah_pos_v1, df_ah_pos_v1)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Odds'] > 1.85))]
    prev = prev[((prev['Over_Line'] >= 200) & (prev['Over_Line'] < 212)) | ((prev['Over_Line'] >= 215) & (prev['Over_Line'] < 218))]

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/ah_pos_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO AH POSITIVO V1 (LR) ------------')
        print()

        prev['Bet'] = 'AH Zebra'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'HA_Line', 'Odds', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO AH POSITIVO V1 (LR) ------------



,Time,League,Home,Away,HA_Line,Odds,prediction_score,Bet
6,08:35,China Cba,Zhejiang Chouzhou,Ningbo Rockets,-20.5,1.87,0.6571,AH Zebra


### AH Zebra V2 (ADA)

In [10]:
# Duplicando o dataset
df_ah_pos_v2 = jogos.copy()

# AH
df_ah_pos_v2.loc[(df_ah_pos_v2['HA_Line']) > 0, 'Zebra'] = 'Casa'
df_ah_pos_v2.loc[(df_ah_pos_v2['HA_Line']) < 0, 'Zebra'] = 'Fora'
df_ah_pos_v2['Zebra'] = df_ah_pos_v2['Zebra'].astype('category')
# Odds
df_ah_pos_v2.loc[(df_ah_pos_v2['Zebra']) == 'Casa', 'Odds'] = df_ah_pos_v2['HA_Odds_H']
df_ah_pos_v2.loc[(df_ah_pos_v2['Zebra']) == 'Fora', 'Odds'] = df_ah_pos_v2['HA_Odds_A']

df_ah_pos_v2.reset_index(drop=True, inplace=True)
df_ah_pos_v2.replace([np.inf, -np.inf], 0, inplace=True)

if not df_ah_pos_v2.empty:
    mdl_ah_pos_v2 = load_model('ML/ah_pos/v2/v2_ah_pos', verbose=False)
    prev = predict_model(mdl_ah_pos_v2, df_ah_pos_v2)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Odds'] >= 1.85))]
    prev = prev[((prev['HA_Line'] <= -7)) & ((prev['HA_Line'] >= -13))]
    
    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/ah_pos_v2_{data_var}.csv', index=False)

        print('------------ MÉTODO AH POSITIVO V2 (ADA) ------------')
        print()

        prev['Bet'] = 'AH Zebra'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'HA_Line', 'Odds', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO AH POSITIVO V2 (ADA) ------------



,Time,League,Home,Away,HA_Line,Odds,prediction_score,Bet
1,23:30,Eua Nba,Los Angeles Clippers,Orlando Magic,-7.5,1.91,0.5004,AH Zebra
7,09:00,China Cba,Xinjiang,Qingdao,-8.5,1.87,0.5445,AH Zebra
15,15:00,Europa Euroliga Feminina,Polkowice F,UNI Gyor F,-7.5,1.87,0.6777,AH Zebra
20,19:30,Brasil Nbb,Pato,Cerrado,-7.5,1.87,0.5993,AH Zebra


### AH Zebra V3 (ADA - SEED 8941)

In [11]:
# Duplicando o dataset
df_ah_pos_v3 = jogos.copy()

# AH
df_ah_pos_v3.loc[(df_ah_pos_v3['HA_Line']) > 0, 'Zebra'] = 'Casa'
df_ah_pos_v3.loc[(df_ah_pos_v3['HA_Line']) < 0, 'Zebra'] = 'Fora'
df_ah_pos_v3['Zebra'] = df_ah_pos_v3['Zebra'].astype('category')
# Odds
df_ah_pos_v3.loc[(df_ah_pos_v3['Zebra']) == 'Casa', 'Odds'] = df_ah_pos_v3['HA_Odds_H']
df_ah_pos_v3.loc[(df_ah_pos_v3['Zebra']) == 'Fora', 'Odds'] = df_ah_pos_v3['HA_Odds_A']

df_ah_pos_v3.reset_index(drop=True, inplace=True)
df_ah_pos_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_ah_pos_v3.empty:
    mdl_ah_pos_v3 = load_model('ML/ah_pos/v3/v3_ah_pos', verbose=False)
    prev = predict_model(mdl_ah_pos_v3, df_ah_pos_v3)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Odds'] >= 1.85))]
    prev = prev[((prev['HA_Line'] <= -7)) & ((prev['HA_Line'] >= -13))]
    
    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/ah_pos_v3_{data_var}.csv', index=False)

        print('------------ MÉTODO AH POSITIVO V3 (ADA - SEED 8941) ------------')
        print()

        prev['Bet'] = 'AH Zebra'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'HA_Line', 'Odds', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO AH POSITIVO V3 (ADA - SEED 8941) ------------



,Time,League,Home,Away,HA_Line,Odds,prediction_score,Bet
1,23:30,Eua Nba,Los Angeles Clippers,Orlando Magic,-7.5,1.91,0.5015,AH Zebra


### AH Zebra V4 (ADA - SEED 4280)

In [12]:
# Duplicando o dataset
df_ah_pos_v4 = jogos.copy()

# AH
df_ah_pos_v4.loc[(df_ah_pos_v4['HA_Line']) > 0, 'Zebra'] = 'Casa'
df_ah_pos_v4.loc[(df_ah_pos_v4['HA_Line']) < 0, 'Zebra'] = 'Fora'
df_ah_pos_v4['Zebra'] = df_ah_pos_v4['Zebra'].astype('category')
# Odds
df_ah_pos_v4.loc[(df_ah_pos_v4['Zebra']) == 'Casa', 'Odds'] = df_ah_pos_v4['HA_Odds_H']
df_ah_pos_v4.loc[(df_ah_pos_v4['Zebra']) == 'Fora', 'Odds'] = df_ah_pos_v4['HA_Odds_A']

df_ah_pos_v4.reset_index(drop=True, inplace=True)
df_ah_pos_v4.replace([np.inf, -np.inf], 0, inplace=True)

if not df_ah_pos_v4.empty:
    mdl_ah_pos_v4 = load_model('ML/ah_pos/v3/v4_ah_pos', verbose=False)
    prev = predict_model(mdl_ah_pos_v4, df_ah_pos_v4)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Odds'] >= 1.85))]
    prev = prev[((prev['HA_Line'] <= -7)) & ((prev['HA_Line'] >= -13))]
    
    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/ah_pos_v4_{data_var}.csv', index=False)

        print('------------ MÉTODO AH POSITIVO V4 (ADA - SEED 4280) ------------')
        print()

        prev['Bet'] = 'AH Zebra'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'HA_Line', 'Odds', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO AH POSITIVO V4 (ADA - SEED 4280) ------------



,Time,League,Home,Away,HA_Line,Odds,prediction_score,Bet
1,23:30,Eua Nba,Los Angeles Clippers,Orlando Magic,-7.5,1.91,0.5026,AH Zebra
15,15:00,Europa Euroliga Feminina,Polkowice F,UNI Gyor F,-7.5,1.87,0.5911,AH Zebra
20,19:30,Brasil Nbb,Pato,Cerrado,-7.5,1.87,0.5911,AH Zebra
